In [1]:
import cv2
import dlib
import numpy as np
from model import Net
import torch
from imutils import face_utils
import winsound as sd
import pygame

pygame.mixer.init()

IMG_SIZE = (34,26)
PATH = "./weights/trained.pth"

device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = Net()
model.load_state_dict(torch.load(PATH,map_location=torch.device('cpu')))
model.eval()

n_count = 0
s_count = 0
y_count = 0
z_count = 0

def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(int)

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect


def predict(pred):
    pred = pred.transpose(1, 3).transpose(2, 3)

    outputs = model(pred)

    pred_tag = torch.round(torch.sigmoid(outputs))

    return pred_tag

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1920)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1080)

def beepsound():
    sound1 = pygame.mixer.Sound("source.mp3")
    sound1.play()

def beepsound2():
    sound2 = pygame.mixer.Sound("source2.mp3")
    sound2.play()


while cap.isOpened():
    ret, img_ori = cap.read()

    if not ret:
        break

    img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])


        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        cv2.imshow('l', eye_img_l)
        cv2.imshow('r', eye_img_r)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32)
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32)


        eye_input_l = torch.from_numpy(eye_input_l)
        eye_input_r = torch.from_numpy(eye_input_r)


        pred_l = predict(eye_input_l)
        pred_r = predict(eye_input_r)

        if pred_l.item() == 0.0 and pred_r.item() == 0.0:
            n_count+=1
                            
        else:
            n_count = 0
           
            
        if n_count > 60:
            cv2.putText(img,"Wake up", (120,160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
            beepsound()
            n_count = 0
            s_count+=1
            y_count=0
        
        else:
            y_count+=1
            
        
        if s_count > 2:
            cv2.putText(img,"YOU ARE IN DROWSINESS", (120,160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
                        
            if y_count > 250:
                s_count=0
                y_count=0

        

    

    # visualize left eye
    if pred_l > 0.1:
        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(0,255,0), thickness=2)
    
    else:
        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(0,0,255), thickness=2)

    # visualize right eye    
    if pred_r > 0.1:
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(0,255,0), thickness=2)
    
    else:
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(0,0,255), thickness=2)
 
    
  #
  # print(state_l)
  # print(state_r)
    cv2.imshow('result', img)
  

    if cv2.waitKey(1) == ord('q'):
        break


pygame 2.3.0 (SDL 2.24.2, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
